# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
from skimage import measure
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [92]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

def load_ants_affine_mat(mat_file_path):
    # Load .mat file

    rotation = np.array([
    [0.941498875617981, 0.19202075898647308, -0.08786671608686447], 
    [-0.198630228638649, 0.781607985496521, 0.015325229614973068],
    [0.06243732571601868, 0.0008880351670086384, 0.8622015118598938]
    ]
    )
    translation = np.array([266.45843505859375, 150.64747619628906, -138.71780395507812])
    center = np.array([730.082275390625, 373.0629577636719, 569.5])
    # Apply rotation around center
    affine = np.eye(4)
    affine[:3, :3] = rotation
    affine[:3, 3] = translation + center - rotation @ center

    return affine

def convert_lps_to_sar(affine_lps):
    # Flip L (x) and P (y) axes: LPS to RAS => flip x and y
    flip = np.diag([-1, -1, 1, 1])
    affine_sar = flip @ affine_lps @ flip
    return affine_sar

In [105]:
moving_name = 'MD594'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
#bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

37


In [99]:
from scipy.io import loadmat
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
transform_file = 'MD594_Allen_10.0x10.0x10.0um_sagittal_to_Allen.mat'
transform_path = os.path.join(reg_path, moving_name, transform_file)
transform = sitk.ReadTransform(transform_path)
#transform = sitk.AffineTransform(transform)
transfo_dict = loadmat(transform_path)
lps2ras = np.diag([-1, -1, 1])
rot = transfo_dict['AffineTransform_float_3_3'][0:9].reshape((3, 3))
trans = transfo_dict['AffineTransform_float_3_3'][9:12]
offset = transfo_dict['fixed']
r_trans = (np.dot(rot, offset) - offset - trans).T * [1, 1, -1]
affine_matrix = np.eye(4)
affine_matrix[0:3, 3] = r_trans
affine_matrix[:3, :3] = np.dot(np.dot(lps2ras, rot), lps2ras)

affine_lps = load_ants_affine_mat(transform_path)
affine_sar = convert_lps_to_sar(affine_lps)

print("Affine Matrix in SAR (RAS) orientation:")
print(affine_lps)

Affine Matrix in SAR (RAS) orientation:
[[ 9.41498876e-01  1.92020759e-01 -8.78667161e-02  2.87573332e+02]
 [-1.98630229e-01  7.81607985e-01  1.53252296e-02  3.68410138e+02]
 [ 6.24373257e-02  8.88035167e-04  8.62201512e-01 -1.06157243e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [106]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 8.98433334e-01 -2.65091877e-01  5.95423255e-02 -2.13381393e+01]
 [ 3.65894141e-01  1.13223847e+00  4.76580505e-02 -6.40295926e+02]
 [-5.71300691e-02 -1.34311696e-02  1.17741204e+00  2.81177915e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [107]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS 238.5831606646421
# MD585 to MD589 RMS 18.2658167690059

RMS 25.0798202104085


In [110]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    x,y,z = moving_all[structure]
    fixed0 = np.array(fixed_all[structure]) 
    #transformed = affine_transform_point(moving0, affine_lps)

    z,y,x = transform.TransformPoint((z,y,x))
    transformed = (x,y,z)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))


RMS 310.6474257036656


In [109]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = affine_transform_point(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')


MD594 SC non trans [1119.  385.  517.]
MD594 SC apply trans [913. 229. 567.]
Allen SC [914. 239. 569.]
MD594->Allen SC [ -1. -10.  -2.]


In [111]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,MD594,Allen,transformed,difference,sumsquares
Index,,,,,,
2,DC_R,"[1305.0, 585.0, 395.0]","[1127.0, 500.0, 816.0]","[1127.0, 633.0, 954.0]","[-0.0, 133.0, 138.0]",191.6526
20,Sp5I_R,"[1305.0, 585.0, 395.0]","[1211.0, 587.0, 764.0]","[1244.0, 694.0, 932.0]","[33.0, 107.0, 168.0]",201.6633
27,LRt_R,"[1305.0, 585.0, 395.0]","[1234.0, 699.0, 699.0]","[1267.0, 776.0, 888.0]","[33.0, 77.0, 189.0]",206.5639
26,Amb_R,"[1305.0, 585.0, 395.0]","[1188.0, 662.0, 706.0]","[1199.0, 761.0, 900.0]","[11.0, 99.0, 194.0]",218.2174
22,Sp5O_R,"[1305.0, 585.0, 395.0]","[1114.0, 591.0, 760.0]","[1171.0, 711.0, 936.0]","[57.0, 119.0, 176.0]",220.3093
6,7N_R,"[1305.0, 585.0, 395.0]","[1085.0, 678.0, 705.0]","[1133.0, 789.0, 903.0]","[48.0, 112.0, 199.0]",232.9344
11,5N_R,"[1305.0, 585.0, 395.0]","[1019.0, 529.0, 729.0]","[1044.0, 713.0, 909.0]","[24.0, 184.0, 180.0]",258.8489
12,Amb_L,"[1305.0, 585.0, 395.0]","[1188.0, 662.0, 432.0]","[1192.0, 803.0, 673.0]","[4.0, 141.0, 241.0]",279.1780
8,LRt_L,"[1305.0, 585.0, 395.0]","[1234.0, 699.0, 439.0]","[1269.0, 824.0, 687.0]","[35.0, 125.0, 248.0]",279.7004


In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
def ants_3d_to_scipy_2d(affine_3d, plane='axial', slice_index=0):
    """
    Convert a 3D ANTs affine transformation matrix to a 2D affine transform
    suitable for scipy.ndimage.affine_transform.

    Parameters:
        affine_3d (np.ndarray): A 4x4 affine matrix from ANTs.
        plane (str): Plane to slice through ('axial', 'coronal', 'sagittal').
        slice_index (int): Index of the slice in the chosen plane.

    Returns:
        matrix_2d (np.ndarray): 2x2 affine transformation matrix.
        offset_2d (np.ndarray): Length-2 offset vector.
    """
    if affine_3d.shape != (4, 4):
        raise ValueError("Expected a 4x4 affine transformation matrix.")

    # Extract rotation+scaling and translation components
    rotation_scaling = affine_3d[:3, :3]
    translation = affine_3d[:3, 3]

    if plane == 'axial':
        matrix_2d = rotation_scaling[:2, :2]
        offset_2d = translation[:2] + rotation_scaling[:2, 2] * slice_index
    elif plane == 'coronal':
        matrix_2d = rotation_scaling[[0,2], :][:, [0,2]]
        offset_2d = translation[[0,2]] + rotation_scaling[[0,2], 1] * slice_index
    elif plane == 'sagittal':
        matrix_2d = rotation_scaling[1:3, 1:3]
        offset_2d = translation[1:3] + rotation_scaling[1:3, 0] * slice_index
    else:
        raise ValueError("Plane must be 'axial', 'coronal', or 'sagittal'.")

    return matrix_2d, offset_2d

In [ ]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
from cloudvolume import CloudVolume
tq = LocalTaskQueue(parallel=2)